# Content Based Model for Movie Recommendation
## Oleh : Febi Andika Dani Fajar Suryawan


## Studi Kasus
Diberikan data berisi 10 judul film beserta genre, director, dan actors dari film tersebut. Akan dibuat daftar rekomendasi film berdasarkan kemiripan suatu film terhadap film lainnya dengan melihat genre, director dan actors film-film tersebut.

## Preparasi Data

In [3]:
# Memanggil library yang dibutuhkan
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
# Memanggil dataset
df=pd.read_csv('film.csv',delimiter=';')
df

,Title,Genre,Director,Actors
0,Ada Apa dengan Cinta 2,"Romance, Drama",Riri Riza,"Nicholas Saputra, Dian Sastrowardoyo, Adinia W..."
1,Gundala,"Action, Drama",Joko Anwar,"Tara Basro, Abimana Aryasatya, Lukman Sardi"
2,Dilan 1991,"Romance, Drama",Pidi Baiq,"Iqbaal Ramadhan, Vanesha Prescilla"
3,Bumi Manusia,"Drama, History",Hanung Bramantyo,"Iqbaal Ramadhan, Mawar Eva"
4,Dua Garis Biru,Drama,Ginatri Noer,"Adhisty Zara, Angga Yunanda"
5,Avengers: End Game,"Action, Fiction",Joe Russo,"Robert Downey, Chris Evans, Chris Hemworth"
6,The Lion King,"Drama, Adventure",Jon Favreau,"Donald Glover, Beyonce"
7,Aladdin,"Romance, Fantasy",Guy Ritchie,"Naomi Scott, Mena Massoud"
8,Spiderman: Far From Home,"Fantasy, Fiction",Jon Watts,"Tom Holland, Zendaya"
9,Captain Marvel,Fiction,Anna Boden,"Brie Larson, Samuel Jackson"


In [5]:
# Mengambil kolom Title, Genre, Director, Actors dari dataset
df=df[['Title','Genre','Director','Actors']]
df.head()

,Title,Genre,Director,Actors
0,Ada Apa dengan Cinta 2,"Romance, Drama",Riri Riza,"Nicholas Saputra, Dian Sastrowardoyo, Adinia W..."
1,Gundala,"Action, Drama",Joko Anwar,"Tara Basro, Abimana Aryasatya, Lukman Sardi"
2,Dilan 1991,"Romance, Drama",Pidi Baiq,"Iqbaal Ramadhan, Vanesha Prescilla"
3,Bumi Manusia,"Drama, History",Hanung Bramantyo,"Iqbaal Ramadhan, Mawar Eva"
4,Dua Garis Biru,Drama,Ginatri Noer,"Adhisty Zara, Angga Yunanda"


In [6]:
#Membuang koma diantara nama lengkap aktor dan mengambil 2 nama awal
df['Actors']=df['Actors'].map(lambda x: x.split(',')[:2])

#Membuang koma diantara Genre
df['Genre']=df['Genre'].map(lambda x: x.lower().split(','))
df['Director']=df['Director'].map(lambda x: x.split(' '))

for index, row in df.iterrows():
    row['Actors'] = [x.lower().replace(' ','') for x in row['Actors']]
    row['Director'] = ''.join(row['Director']).lower()

In [7]:
# Membuat kolom Title menjadi index
df.set_index('Title', inplace = True)
df.head()

,Genre,Director,Actors
Title,,,
Ada Apa dengan Cinta 2,"[romance, drama]",riririza,"[nicholassaputra, diansastrowardoyo]"
Gundala,"[action, drama]",jokoanwar,"[tarabasro, abimanaaryasatya]"
Dilan 1991,"[romance, drama]",pidibaiq,"[iqbaalramadhan, vaneshaprescilla]"
Bumi Manusia,"[drama, history]",hanungbramantyo,"[iqbaalramadhan, mawareva]"
Dua Garis Biru,[drama],ginatrinoer,"[adhistyzara, anggayunanda]"


In [8]:
# Menggabungkan kolom Genre, Director, Actors menjadi satu 
df['words'] = ''
columns = df.columns
for index, row in df.iterrows():
    words = ''
    for col in columns:
        if col != 'Director':
            words = words + ' '.join(row[col])+ ' '
        else:
            words = words + row[col]+ ' '
    row['words'] = words
    
df.drop(columns = [col for col in df.columns if col!= 'words'], inplace = True)
df.head()

,words
Title,
Ada Apa dengan Cinta 2,romance drama riririza nicholassaputra diansa...
Gundala,action drama jokoanwar tarabasro abimanaaryas...
Dilan 1991,romance drama pidibaiq iqbaalramadhan vanesha...
Bumi Manusia,drama history hanungbramantyo iqbaalramadhan ...
Dua Garis Biru,drama ginatrinoer adhistyzara anggayunanda


## Content Based Model

Dalam membuat model rekomendasi film berdasarkan konten, akan digunakan cosine similarity matrix untuk menghitung kemiripan diantara tiap film. Untuk menghitung cosine similarity matrix digunakan fungsi CountVectorizer untuk membangkitkan count matrix yang berisi jumlah kata dalam kolom words setiap baris berdasarkan daftar kata yang unik dari kumpulan kata di kolom words.

In [9]:
# Membangkitkan count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df['words'])

In [10]:
# Membangun cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.       , 0.2      , 0.4      , 0.2      , 0.2236068, 0.       ,
        0.2      , 0.2      , 0.       , 0.       ],
       [0.2      , 1.       , 0.2      , 0.2      , 0.2236068, 0.2      ,
        0.2      , 0.       , 0.       , 0.       ],
       [0.4      , 0.2      , 1.       , 0.4      , 0.2236068, 0.       ,
        0.2      , 0.2      , 0.       , 0.       ],
       [0.2      , 0.2      , 0.4      , 1.       , 0.2236068, 0.       ,
        0.2      , 0.       , 0.       , 0.       ],
       [0.2236068, 0.2236068, 0.2236068, 0.2236068, 1.       , 0.       ,
        0.2236068, 0.       , 0.       , 0.       ],
       [0.       , 0.2      , 0.       , 0.       , 0.       , 1.       ,
        0.       , 0.       , 0.2      , 0.2236068],
       [0.2      , 0.2      , 0.2      , 0.2      , 0.2236068, 0.       ,
        1.       , 0.       , 0.       , 0.       ],
       [0.2      , 0.       , 0.2      , 0.       , 0.       , 0.       ,
        0.       , 1.       , 0.2    

Selanjutnya, dibuat fungsi untuk membuat daftar film berdasarkan kemiripannya dengan film lain. Misalkan terdapat film A, maka akan direkomendasikan daftar 3 film yang memiliki kemiripan paling tinggi dengan film A berdasarkan cosine similarity matrix yang telah dibuat.

In [11]:
# fungsi untuk mengambil judul film sebagai input dan mengeluarkan 3 rekomendasi film 
indices = pd.Series(df.index)
def recommendations(title, cosine_sim = cosine_sim):
    
    recommended_movies = []
    # Mengambil indeks film yang sesuai dengan judul film
    idx = indices[indices == title].index[0]

    # Membuat series yang berisi similarity scores
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # Mengambil indeks dari 3 film yang memiliki kemiripan paling tinggi
    top_10_indexes = list(score_series.iloc[1:4].index)
    
    # Membuat list berisi judul film yang direkomendasikan
    for i in top_10_indexes:
        recommended_movies.append(list(df.index)[i])
        
    return recommended_movies

Sebagai contoh, akan dicari rekomendasi film yang memiliki kemiripan dengan film Captain Marvel sebagai berikut.

In [12]:
# Input judul film (Input terdapat pada data)
title=input('Masukkan judul film:')

# Memanggil fungsi untuk mengeluarkan 3 rekomendasi film
recommendation=pd.DataFrame(recommendations(title),columns=['Movie Title'])

# Mengeluarkan 3 rekomendasi film yang similar
print('Judul film yang similar dengan film {} yang direkomendasikan antara lain:'.format(title))
print(recommendation)

Masukkan judul film:Captain Marvel
Judul film yang similar dengan film Captain Marvel yang direkomendasikan antara lain:
                Movie Title
0  Spiderman: Far From Home
1        Avengers: End Game
2                   Aladdin


**Catatan :** Pada content based model, alangkah baiknya digunakan data film yang relatif lebih banyak. Hal ini dapat membuat proses melihat kemiripan suatu film jauh lebih akurat dan baik (sudah dibuktikan). Pada model ini hanya digunakan data yang telah dibuat dalam proses pembelajaran kelas saja.